<a href="https://colab.research.google.com/github/R0b0t-Maker/LLM-T/blob/main/chatbot_LangChain_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pinecone**

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
!pip install -r "/content/drive/MyDrive/Udemy/LangChain/p-1/requirements.txt" -q
!pip install --quiet openai python-dotenv
!pip install -q pinecone-client


In [61]:
!pip install --upgrade -q pinecone-client

In [62]:
!pip show pinecone-client

Name: pinecone-client
Version: 4.0.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: 


In [63]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv("/content/drive/MyDrive/Udemy/LangChain/p-1/env", override=True)

True

In [64]:
from pinecone import Pinecone

pc=Pinecone()
# pc=Pinecone(api_key='e4179379-10db-4ee7-9082-a01a78cda928')
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'langchain-apju9rk.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [65]:
pc.list_indexes()[0]

{'dimension': 1536,
 'host': 'langchain-apju9rk.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'langchain',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [66]:
pc.list_indexes()[0]['name']

'langchain'

In [68]:
pc.describe_index('quickstart')

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2024-04', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '09d46ce9c3833a1c45cee1fcfdbe7372', 'Date': 'Mon, 13 May 2024 15:35:05 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource quickstart not found"},"status":404}


In [69]:
from pinecone import ServerlessSpec

index_name= 'langchain'
if index_name not in pc.list_indexes().names():
  print(f'Creating index:{index_name}')
  pc.create_index(
      name= index_name,
      dimension=1536,
      metric='cosine',
      spec=ServerlessSpec(
          cloud = 'aws',
          region='us-east-1'
      )

  )
  print('Index created!')
else:
  print(f'Index{index_name} already exsits!')

Indexlangchain already exsits!


In [70]:
index_name='quickstart'
if index_name in pc.list_indexes().names():
  print(f'Deleting index{index_name}...')
  pc.delete_index(index_name)
  print('done!!!')
else:
  print(f'Index {index_name} does not exist!')

Index quickstart does not exist!


In [71]:
index_name= 'langchain'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [72]:
#inserting vectors
import random

vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('abcde')

index_nmae = 'langchain'
index = pc.Index(index_name)

index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [73]:
#updating vectors
index.upsert(vectors=[('c', [0.5]*1536)])
#fetching vectors

index.fetch(ids=['c','d'])

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'c': {'id': 'c',
                   'values': [0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
             

In [74]:
# #deleting vectors
# index.delete(ids=['b', 'a'])

In [75]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [76]:
index.fetch(ids=['x'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [77]:
#query
query_vector = [random.random() for _ in range(1536)]
index.query(
    vector=query_vector,
    top_k=3,
    include_values=False
)

{'matches': [{'id': 'c', 'score': 0.864346743, 'values': []},
             {'id': 'e', 'score': 0.755693197, 'values': []},
             {'id': 'd', 'score': 0.748294, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}